In [3]:
# Using k-fold validation
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [9]:
kfold = KFold(n_splits = 3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 :\n',label_train.value_counts())
    print('검증 레이블 데이터 분포 :\n',label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 :
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    50
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 :
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    50
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 :
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    50
Name: label, dtype: int64


In [11]:
# Using Stratidied Kfold

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 :\n',label_train.value_counts())
    print('검증 레이블 데이터 분포 :\n',label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 :
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 :
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 :
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    16
1    16
0    16
Name: label, dtype: int64


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

dt_clf = DecisionTreeClassifier(random_state=156)
skfold = StratifiedKFold(n_splits = 3)
n_iter = 0
cv_accuracy = []

features = iris.data
label = iris.target

for train_index, test_index in skfold.split(features, label):
    # spliting data sets using stratified k-fold system
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # fitting and making prediction
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # checking accuracy
    n_iter += 1
    accuracy= np.around(accuracy_score(y_test, pred), 4)
    print('\n## {0} 교차 검증\n 정확도 : {1}'.format(n_iter, accuracy))
    cv_accuracy.append(accuracy)
    
print('\n 교차 검증 정확도 :', np.round(cv_accuracy, 4))
print('\n 평균 검증 정확도 :', np.mean(cv_accuracy))


## 1 교차 검증
 정확도 : 0.9804

## 2 교차 검증
 정확도 : 0.9216

## 3 교차 검증
 정확도 : 0.9792

 교차 검증 정확도 : [0.9804 0.9216 0.9792]

 평균 검증 정확도 : 0.9604


In [21]:
# Using GridSearchCV

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2, random_state=121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]}

import pandas as pd

# hyper parameter of param_grid : 3개의 train, test fold로 나누어 test 수행
# refit = true가 default
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3)
grid_dtree.fit(X_train, y_train)

# Extracting results
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df

print('GridSearchCV 최적 파라미터 :', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 :', grid_dtree.best_score_)

estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print('\n 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 최적 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.975

 정확도 : 0.9667
